__Data notes__
- COVID cases
    - one file per measure (confirmed dealths, recovered)
    - one row per province/country
    - no duplicates from latlongs or multiple entries in same day
        - however, latlongs have dodgy numbers of digits so not very accurate: prob wont use these!
    - dates are columns, with new col added each day and zeros for missing data

Ideas
- add population data per country
- NOTE: lat/long were added to daily data on march 1st: jan and feb tables only have province/country!


# Imports and functions

In [2]:
import logging
import glob
import pandas as pd
import os
import geopandas as gpd
import json 
import datetime
import itertools
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.models import DateSlider, Select, HoverTool, DatetimeTickFormatter, NumeralTickFormatter
from bokeh.layouts import widgetbox, row, column
from bokeh.models.annotations import Title
from bokeh.models.widgets import Panel, Tabs
import colorcet 

In [9]:
os.chdir('/Users/jdorni/Documents/training/covid-19-bokeh/')

# Functions

In [80]:
def prepare_data(df, metric):
    """Clean and subset df by metric based on user input
    
    Args:
        df {pd.DataFrame} one row per country/date with country geometry and a column for each metric
        metric {str} one of ['Confirmed', 'Deaths', 'Recovered']
        
    Returns:
        df {pd.DataFrame}
    """
    
    # organise cols
    df.rename({'Country/Region': 'country', 'Province/State': 'province'}, axis=1, inplace=True)
    df.columns = [x.lower() for x in df.columns]
    df.fillna('No data', inplace=True)
    
    # convert from wide to long 
    # to get one row per province/country/date
    df = pd.melt(df, id_vars=['country', 'province', 'lat', 'long'],
                 var_name='day', value_name=metric)
    df.set_index(['country', 'province', 'lat', 'long', 'day'], inplace=True)
    
    return df.sort_values(by=['country', 'province'])

In [6]:
def preprocess():
    """
    Prepare df for plotting
    
    - imports COVID-19 case and country boundary datasets
    - validates country names
    - joins datasets together
    - sets datatime data types
    - adds missing rows for countries with no cases reported on certain dates
    """
    
    ## import data

    confirmed = pd.read_csv('data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    deaths = pd.read_csv('data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
    recovered = pd.read_csv('data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

    # read in shapefile using Geopandas
    shapefile = 'data/country_boundaries/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]


    ## prepare data

    # rename columns
    gdf.columns = ['country', 'country_code', 'geometry']

    confirmed_melted = prepare_data(df=confirmed, metric='confirmed')
    deaths_melted = prepare_data(df=deaths, metric='deaths')
    recovered_melted = prepare_data(df=recovered, metric='recovered')

    df = pd.concat([confirmed_melted, deaths_melted, recovered_melted], axis=1)
    df.reset_index(inplace=True)


    # sum provinces to get totals by whole countries  
    df = df.groupby(['country', 'day']).agg(
        confirmed=pd.NamedAgg(column='confirmed', aggfunc='sum'),
        deaths=pd.NamedAgg(column='deaths', aggfunc='sum'),
        recovered=pd.NamedAgg(column='recovered', aggfunc='sum'))

    df = df.reset_index()

    # convert date to a sortable string format
    df['day'] = pd.to_datetime(df['day']).dt.strftime("%Y-%m-%d")


    # keep record of original country name
    df['country_original'] = df['country']

    # remap country names (need to sum again after this as some countries are combined)

    country_map = {
        'Congo (Kinshasa)': 'Democratic Republic of the Congo',
        'Congo (Brazzaville)': 'Democratic Republic of the Congo',
        "Cote d'Ivoire": 'Ivory Coast',
        'Eswatini': 'eSwatini',
        'Gambia, The': 'Gambia', 
        'The Gambia': 'Gambia', 
        'Korea, South': 'South Korea', 
        'North Macedonia': 'Macedonia', 
        'Serbia': 'Republic of Serbia',
        'Taiwan*': 'Taiwan',
        'Tanzania': 'United Republic of Tanzania',
        'Timor-Leste': 'East Timor',
        'Bahamas, The': 'The Bahamas',
        'US': 'United States of America',
        'Holy See': 'Italy'    # church jurisdiction and not country (vatican)
    }

    df.replace({"country": country_map}, inplace=True)


    # left join so only countries we have geometries for are in the mapping dataset
    merged = gdf.merge(df, left_on = 'country', right_on = 'country', how='left')
    merged = merged.drop(['country_code'], axis=1)

    # re-convert day (including the NaN rows) to the correct string date format 
    merged['day'] = pd.to_datetime(merged['day']).dt.strftime("%Y-%m-%d")

    ## Add missing date rows for countries with no cases

    # create tmp table of all combinations of country X date
    country_date_cols = ['country', 'day']
    lists_of_uniques = [merged[col].unique() for col in country_date_cols]
    tmp = pd.DataFrame(list(itertools.product(*lists_of_uniques)), columns=country_date_cols)

    # add geometry from country borders
    tmp = tmp.merge(gdf[['country', 'geometry']], left_on = 'country', right_on = 'country', how='left')

    # outer join to add missing rows - will join on common columns
    merged = merged.merge(tmp, how='outer')

    # del rows with day='NaT' and NaNs
    merged = merged.loc[merged['day'] != 'NaT']
    merged.dropna(subset=['day'], inplace=True) #XXX CHECK THIS IS THE RIGHT THING TO DO!!

    # sort by date to plot time series
    merged.sort_values(by=['country', 'country_original', 'day'], inplace=True)
    
    return merged

In [12]:
def source_by_date(data, selected_day):
    """Create geosource for date selected by user on slider. Returns pd.DataFrame."""
    selected_day = selected_day.strftime('%Y-%m-%d')
    new_data = data.loc[data['day'] == selected_day] 
    return new_data

In [15]:
# Bokeh functions

def slider_callback(attr, old, new):
    """Update numbers based on selected date"""
    day = date_slider.value

    # hack to convert bokeh epoch unix time to proper date: 
    # bokeh misses the decimal point so we must divide by 1000 - Antonio logged an issue
    day = datetime.datetime.fromtimestamp(day/1000)

    new_data = source_by_date(data, day)
    source.geojson = new_data.to_json() # overwrite the existing source's geojson

    
def menu_callback(attr, old, new):
    """Update color shading by selected metric"""
    
    if menu.value == 'Confirmed': 
        metric = 'confirmed'  
        color_mapper.palette = colorcet.b_linear_blue_5_95_c73[::-1]
        tooltips=[('Country', '@country'), ('Confirmed', '@confirmed{0,0}')]
        
    elif menu.value == 'Deaths':
        metric = 'deaths'
        color_mapper.palette = colorcet.b_linear_kry_5_98_c75[::-1]
        tooltips=[('Country', '@country'), ('Deaths', '@deaths{0,0}')]
        
    elif menu.value == 'Recovered': 
        metric = 'recovered'
        color_mapper.palette = colorcet.b_linear_green_5_95_c69[::-1]
        tooltips=[('Country', '@country'), ('Recovered', '@recovered{0,0}')]
        
    else:
        print('Unknown value')
        
    # update tooltips
    p1.hover.tooltips = tooltips
    
    # update colours
    vals = data[metric]    
    color_mapper.low = vals.min()
    color_mapper.high = vals.max()
    country_polygons.glyph.fill_color = {'field': metric, 'transform': color_mapper}
    color_bar.color_mapper = color_mapper

# Prepare data

In [10]:
data = preprocess()

In [11]:
data.head()

,country,geometry,day,confirmed,deaths,recovered,country_original
8660,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",2020-01-22,0.0,0.0,0.0,Afghanistan
8661,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",2020-01-23,0.0,0.0,0.0,Afghanistan
8662,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",2020-01-24,0.0,0.0,0.0,Afghanistan
8663,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",2020-01-25,0.0,0.0,0.0,Afghanistan
8664,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",2020-01-26,0.0,0.0,0.0,Afghanistan


# Explore World Population data

Total population by sex, annually from 1950 to 2100. Encoded in UTF-8.

Columns:
- LocID (numeric): numeric code for the location; for countries and areas, it follows the ISO 3166-1 numeric standard
- Location (string): name of the region, subregion, country or area
- VarID (numeric): numeric code for the variant 
- Variant (string): projection variant name (Medium is the most used); for more information see https://population.un.org/wpp/DefinitionOfProjectionVariants/
- Time (string): label identifying the single year (e.g. 1950) or the period of the data (e.g. 1950-1955)
- MidPeriod (numeric): numeric value identifying the mid period of the data, with the decimal representing the month (e.g. 1950.5 for July 1950)
- PopMale: Total male population (thousands)
- PopFemale: Total female population (thousands)
- PopTotal: Total population, both sexes (thousands)
- PopDensity: Population per square kilometre (thousands)




In [21]:
ls data/populations/

WPP2019_TotalPopulationBySex.csv


In [149]:
population = pd.read_csv('data/populations/WPP2019_TotalPopulationBySex.csv')

In [150]:
population = population.loc[(population['Time'] == 2020) & (population['Variant'] == 'Medium')]

In [151]:
population.LocID.nunique(), population.Location.nunique()

(477, 474)

In [61]:
population.groupby('Location')['LocID'].nunique().reset_index().sort_values(by='LocID', ascending=False)

,Location,LocID
237,Latin America and the Caribbean,2
171,Europe,2
312,Northern America,2
311,Northern Africa and Western Asia,1
323,Organization of the Islamic Conference (OIC),1
...,...,...
154,Eastern Africa,1
153,East African Community (EAC),1
152,ESCWA: member countries,1
151,ESCWA: Mashreq countries,1


In [64]:
population.loc[population['Location'] == 'Latin America and the Caribbean']
population.loc[population['Location'] == 'Europe']
population.loc[population['Location'] == 'Northern America']

,LocID,Location,PopMale,PopFemale,PopTotal,PopDensity
184625,918,Northern America,182580.522,186289.122,368869.644,19.777
184626,905,Northern America,182580.522,186289.122,368869.644,19.777


In [65]:
population.drop(['LocID', 'Time', 'Variant', 'VarID', 'MidPeriod'], axis=1, inplace=True, errors='ignore')

In [70]:
population.drop_duplicates(inplace=True)

In [74]:
population.loc[population['Location'] == 'Latin America and the Caribbean']
population.loc[population['Location'] == 'Europe']
population.loc[population['Location'] == 'Northern America']

,Location,PopMale,PopFemale,PopTotal,PopDensity
184625,Northern America,182580.522,186289.122,368869.644,19.777


In [ ]:
# NOTE LocID is the standard 3 digit country code. Can i join on this to the country boundaries dataset?

In [153]:
population.loc[population['LocID'] == 834]

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
262523,834,United Republic of Tanzania,2,Medium,2020,2020.5,29851.108,29883.105,59734.213,67.435


In [159]:
# read in shapefile using Geopandas
shapefile = 'data/country_boundaries/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3','ISO_N3','geometry']]


## prepare data

# rename columns
gdf.columns = ['country', 'country_code', 'iso_n3', 'geometry']

In [160]:
gdf['iso_n3'] = gdf['iso_n3'].astype(int)

In [154]:
len(gdf), len(population)

(177, 477)

In [163]:
len(population.merge(gdf, left_on='LocID', right_on='iso_n3', how='left'))

477

In [164]:
len(gdf.merge(population, right_on='LocID', left_on='iso_n3', how='left'))

177

In [174]:
merged = gdf.merge(population, right_on='LocID', left_on='iso_n3', how='left')
merged.head()

,country,country_code,iso_n3,geometry,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
0,Fiji,FJI,242,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",242.0,Fiji,2.0,Medium,2020.0,2020.5,454.018,442.426,896.444,49.066
1,United Republic of Tanzania,TZA,834,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",834.0,United Republic of Tanzania,2.0,Medium,2020.0,2020.5,29851.108,29883.105,59734.213,67.435
2,Western Sahara,SAH,732,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",732.0,Western Sahara,2.0,Medium,2020.0,2020.5,312.260,285.070,597.330,2.246
3,Canada,CAN,124,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",124.0,Canada,2.0,Medium,2020.0,2020.5,18732.178,19009.979,37742.157,4.150
4,United States of America,USA,840,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",840.0,United States of America,2.0,Medium,2020.0,2020.5,163786.016,167216.631,331002.647,36.185


In [208]:
len(merged[['country_code', 'iso_n3', 'PopTotal']].drop_duplicates()) #, 'geometry'

177

In [209]:
merged.groupby(['iso_n3'])['country_code'].nunique().reset_index().sort_values(by='country_code', ascending=False)

,iso_n3,country_code
0,-99,4
119,586,1
111,524,1
112,528,1
113,540,1
114,548,1
115,554,1
116,558,1
117,562,1
118,566,1


In [213]:
merged.loc[merged['iso_n3'] == -99]

,country,country_code,iso_n3,geometry,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
21,Norway,NOR,-99,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,Northern Cyprus,CYN,-99,"POLYGON ((32.73178 35.14003, 32.80247 35.14550...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,Somaliland,SOL,-99,"POLYGON ((48.94820 11.41062, 48.94820 11.41062...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,Kosovo,KOS,-99,"POLYGON ((20.59025 41.85541, 20.52295 42.21787...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:
gdf.loc[gdf['country']=='Norway']

,country,country_code,iso_n3,geometry
21,Norway,NOR,-99,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."


In [219]:
population.loc[population['Location']=='Norway']

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
188091,578,Norway,2,Medium,2020,2020.5,2739.983,2681.259,5421.242,14.842


In [225]:
population.loc[population['Location']=='Cyprus']

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
60767,196,Cyprus,2,Medium,2020,2020.5,603.514,603.847,1207.361,130.667


In [223]:
gdf.loc[gdf['country']=='Northern Cyprus']

,country,country_code,iso_n3,geometry
160,Northern Cyprus,CYN,-99,"POLYGON ((32.73178 35.14003, 32.80247 35.14550..."


In [229]:
gdf.country.nunique(), population.LocID.nunique()

(177, 477)

In [ ]:
#xxx up to here fri 25/04/2020

In [140]:
pd.set_option('display.max_rows', 200)
gdf.loc[gdf['NAME'] == 'Congo'].T

,67
featurecla,Admin-0 country
scalerank,1
LABELRANK,4
SOVEREIGNT,Republic of the Congo
SOV_A3,COG
ADM0_DIF,0
LEVEL,2
TYPE,Sovereign country
ADMIN,Republic of the Congo
ADM0_A3,COG


In [141]:
gdf.ISO_N3.nunique()

174

In [168]:
# look at covid data to see if there are isocodes or countries

df = pd.read_csv('data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df.rename({'Country/Region': 'country', 'Province/State': 'province'}, axis=1, inplace=True)
df.columns = [x.lower() for x in df.columns]
df.fillna('No data', inplace=True)
df.head()

,province,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20
0,No data,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,714,784,840,906,933,996,1026,1092,1176,1279
1,No data,Albania,41.1533,20.1683,0,0,0,0,0,0,...,475,494,518,539,548,562,584,609,634,663
2,No data,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2070,2160,2268,2418,2534,2629,2718,2811,2910,3007
3,No data,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,659,673,673,696,704,713,717,717,723,723
4,No data,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,19,19,19,19,24,24,24,24,25,25


In [169]:
df.country.nunique()

185

# Plot

In [14]:
from bokeh.models.widgets import Panel, Tabs

tab1 = Panel(child=p1, title='Global Cases') # world map to show all countries, with date slider and metric toggle
tab2 = Panel(child=p2, title='Country Selector') # drop down to select country, with time series bar chart for each metric, and possibly table of data by province, and map of data by province

layout = Tabs(tabs=[tab1, tab2])
output_file('tabs.html')
show(layout)

NameError: name 'p1' is not defined

# PLOT 5: Covid data only (points, no map)

In [35]:
## points coloured by country (individual reports)

data = df.copy()
source = ColumnDataSource(data) # convert df to dict of colname: values (note all cols must be same length!)

# Make a CategoricalColorMapper 
n = data['country'].nunique()
print(n, len(data))
palette = inferno(n)
color_mapper = CategoricalColorMapper(factors=data['country'].unique(), palette=palette)

TOOLTIPS = [
    ('Country', '@country'),
    ('Province', '@province'),
    ('Confirmed', '@Confirmed'),
    ('Deaths', '@Deaths')
]

p = figure(plot_width=900, plot_height=500, title='test', tooltips=TOOLTIPS,
           x_axis_label='Longitude', y_axis_label='Latitude')
#p.hover.point_policy = "follow_mouse"

p.circle('Longitude','Latitude', source=source, 
         color=dict(field='country', transform=color_mapper), size=5)

output_notebook()
show(p)

182 14168


NameError: name 'inferno' is not defined

In [57]:
# sum provinces to get totals by whole countries  

df_agg = df.groupby('country').agg(
    confirmed=pd.NamedAgg(column='Confirmed', aggfunc='sum'),
    deaths=pd.NamedAgg(column='Deaths', aggfunc='sum'),
    recovered=pd.NamedAgg(column='Recovered', aggfunc='sum'),
    Latitude=pd.NamedAgg(column='Latitude', aggfunc='mean'),
    Longitude=pd.NamedAgg(column='Longitude', aggfunc='mean'),
    n_rows=pd.NamedAgg(column='Last Update', aggfunc='count'))

df_agg = df_agg.reset_index()
df_agg.head(6)

,Country/Region,confirmed,deaths,recovered,Latitude,Longitude,n_rows
0,Afghanistan,22,0,1,33.9391,67.7100,1
1,Albania,59,2,0,41.1533,20.1683,1
2,Algeria,74,7,12,28.0339,1.6596,1
3,Andorra,39,0,1,42.5063,1.5218,1
4,Antigua and Barbuda,1,0,0,17.0608,-61.7964,1
5,Argentina,79,2,3,-38.4161,-63.6167,1


In [72]:
# points coloured by number of cases (in prog)


data = df_agg.copy()
source = ColumnDataSource(data)

# Create a ColumnDataSource from df: source
source = ColumnDataSource(data)  
 
# Make a CategoricalColorMapper 
n = data['Country/Region'].nunique()
print(n, len(data))
palette = inferno(n)
color_mapper = CategoricalColorMapper(factors=data['Country/Region'].unique(), palette=palette)

p = figure(plot_width=900, plot_height=500, title='test', x_axis_label='Longitude', y_axis_label='Latitude')

p.circle('Longitude','Latitude', source=source, 
         color=dict(field='Country/Region', transform=color_mapper), size=5)

output_notebook()
show(p)

163 163


Loading BokehJS ...

# Time series

In [36]:
# source for plotting bars

# subset by country
country_key = 'United Kingdom'
merged_subset = merged.loc[merged['country'] == country_key].copy()

# convert day strings to datetimes for plotting as time series
merged_subset['day'] = pd.to_datetime(merged_subset['day'])
merged_subset.drop('geometry', axis=1, inplace=True)
merged_subset.head()

,country,day,confirmed,deaths,recovered,country_original
10025,United Kingdom,2020-01-22,0.0,0.0,0.0,United Kingdom
10026,United Kingdom,2020-01-23,0.0,0.0,0.0,United Kingdom
10027,United Kingdom,2020-01-24,0.0,0.0,0.0,United Kingdom
10028,United Kingdom,2020-01-25,0.0,0.0,0.0,United Kingdom
10029,United Kingdom,2020-01-26,0.0,0.0,0.0,United Kingdom


In [38]:
#source = ColumnDataSource(merged_subset) # coldatasources must exclude geometry column
df = merged_subset

metric = 'confirmed'

p = figure(title='Country', plot_height=400 , plot_width=850, 
           toolbar_location='right', 
           tools='wheel_zoom, pan, reset', 
           x_axis_label='Date', y_axis_label='Count',
        x_axis_type='datetime')

# p.xgrid.grid_line_color = None
# p.ygrid.grid_line_color = None

p.line(df['day'], df[metric])

# format the date on x axis
p.xaxis.formatter = DatetimeTickFormatter(days=["%d %b"])
    
# add thousand-separator to y axis
p.yaxis.formatter=NumeralTickFormatter() 


# Create a HoverTool: hover
hover = HoverTool(tooltips=TOOLTIPS)

# Add the hover tool to the figure p
p.add_tools(hover)


output_notebook()
show(p)

Loading BokehJS ...

# Map by province

In [51]:
# # source for mapping by province #TODO - need to unaggregate by province

#merged_subset = merged.loc[merged['country'] == 'United Kingdom']

# source = merged_subset.copy()
# source = GeoJSONDataSource(geojson=json.dumps(json.loads(source.to_json()))) 